In [ ]:
# [예제 2.25] 기본 설정

import pandas as pd
import xlwings as xw

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
# 맥OS인 경우 위 두 줄을 입력하지 말고 아래 코드를 입력하세요
# rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# [예제 2.26] 소득 데이터 읽어오기 

#path = r'C:\Users\JK\Desktop\부동산 데이터\행정구역_시도_별_1인당_지역내총생산__지역총소득__개인소득_20180821155737.xlsx'
path = r'내려받은 개인소득 엑셀파일의 디렉터리\내려받은 개인소득 엑셀 파일명.xlsx'
raw = pd.read_excel(path, sheet_name='데이터', index_col=0)

raw

In [ ]:
# [예제 2.27] 첫 번째 행 제거

raw.drop(['행정구역(시도)별'], inplace=True)

In [ ]:
# [예제 2.28] 인덱스의 시도 이름을 약어로 변경하기 

index_list = raw.index
new_index = []
for temp in index_list:
    if temp[-1] == '시':
        new_index.append(temp[:2])
    elif temp[-1] == '도':
        if len(temp) == 3:
            new_index.append(temp[:2])
        elif len(temp) == 4:
            new_index.append(temp[0] + temp[2])
        else:
            new_index.append('제주')
    else:
        new_index.append(temp)
        
raw.index = new_index

In [ ]:
# [예제 2.29] 컬럼과 인덱스 서로 바꾸기 

income_data = raw.T

income_data

In [ ]:
# [예제 2.30] 소득 데이터 전처리 과정 함수화 

def income_preprocessing(path):
    # path : KB 데이터 엑셀 파일의 디렉토리 (문자열)
    
    raw = pd.read_excel(path)
    raw.drop([0], inplace=True)
    raw.set_index('행정구역(시도)별', inplace=True)
    index_list = raw.index
    new_index = []
    for temp in index_list:
        if temp[-1] == '시':
            new_index.append(temp[:2])
        elif temp[-1] == '도':
            if len(temp) == 3:
                new_index.append(temp[:2])
            elif len(temp) == 4:
                new_index.append(temp[0] + temp[2])
            else:
                new_index.append('제주')
        else:
            new_index.append(temp)

    raw.index = new_index
    income_data = raw.T
    return income_data


income_data_path = r'내려받은 개인소득 엑셀파일의 디렉터리\내려받은 개인소득 엑셀 파일명.xlsx'
income_data = income_preprocessing(income_data_path)

In [ ]:
# [예제 2.31] KB 가격지수 데이터 가져오기 

def KBpriceindex_preprocessing(path, data_type):
    # path : KB 데이터 엑셀 파일의 디렉토리 (문자열)
    # data_type : ‘매매종합’, ‘매매APT’, ‘매매연립’, ‘매매단독’, ‘전세종합’, ‘전세APT’, ‘전세연립’, ‘전세단독’ 중 하나
    
    wb = xw.Book(path)                
    sheet = wb.sheets[data_type]   
    row_num = sheet.range(1,1).end('down').end('down').end('down').row  
    data_range = 'A2:GE' + str(row_num)
    raw_data = sheet[data_range].options(pd.DataFrame, index=False, header=True).value 
    
    bignames = '서울 대구 부산 대전 광주 인천 울산 세종 경기 강원 충북 충남 전북 전남 경북 경남 제주도 6개광역시 5개광역시 수도권 기타지방 구분 전국'
    bigname_list = bignames.split(' ')
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])

    for num, gu_data in enumerate(small_col):
        if gu_data == None:
            small_col[num] = big_col[num]

        check = num
        while True:
            if big_col[check] in bigname_list:
                big_col[num] = big_col[check]
                break
            else:
                check = check - 1
                
    big_col[129] = '경기' 
    big_col[130] = '경기'
    small_col[185] = '서귀포'
    
    raw_data.columns = [big_col, small_col]
    new_col_data = raw_data.drop([0,1])
    
    index_list = list(new_col_data['구분']['구분'])

    new_index = []

    for num, raw_index in enumerate(index_list):
        temp = str(raw_index).split('.')
        if int(temp[0]) > 12 :
            if len(temp[0]) == 2:
                new_index.append('19' + temp[0] + '.' + temp[1])
            else:
                new_index.append(temp[0] + '.' + temp[1])
        else:
            new_index.append(new_index[num-1].split('.')[0] + '.' + temp[0])

    new_col_data.set_index(pd.to_datetime(new_index), inplace=True)
    cleaned_data  = new_col_data.drop(('구분', '구분'), axis=1)
    return cleaned_data



path = r' 여러분이 내려 받은 KB 엑셀파일의 디렉터리를 넣으세요 \ KB엑셀 파일명.xls'
data_type = '매매종합'
price_data = KBpriceindex_preprocessing(path, data_type)

In [ ]:
# [예제 2.32] 특정 지역들의 부동산 가격지수 변화와 개인소득 변화를 subplot으로 나타내기 

location_list = ['전국', '서울', '부산', '대구', '대전' ,'광주', '경기']
start_year = '2004'
end_year = '2016'

num_row = int((len(location_list)-1)/2)+1
plt.figure(figsize=(12, num_row*5))
for j, location in enumerate(location_list):
    year_data = []
    for i in range(int(start_year), int(end_year) + 1):
        if location == '제주도':
            year_data.append(price_data[location]['서귀포'][str(i)+'.12.1'])
        else:
            year_data.append(price_data[location][location][str(i)+'.12.1'])

    temp_df = pd.DataFrame(income_data[location][start_year:end_year])
    temp_df.columns = [location + '소득']
    temp_df[location + '부동산 가격지수'] = year_data
    temp_df['소득 변화율'] = (temp_df[location + '소득']/temp_df[location + '소득'][0] - 1 )*100
    temp_df['부동산 기격 지수 변화율'] = (temp_df[location + '부동산 가격지수']/temp_df[location + '부동산 가격지수'][0] - 1 )*100

    plt.subplot(num_row, 2, j+1)
    plt.title(location + ', ' + start_year + ' ~ ' + end_year + '까지')
    plt.plot(temp_df['부동산 기격 지수 변화율'], label=location + ' 부동산 가격 지수 변화율')
    plt.plot(temp_df['소득 변화율'], label=location + ' 소득 변화율')
    plt.legend()

In [ ]:
# [예제 2.33] PIR 데이터 가져오기 (ch02/ 2.2 ‘기준’이 필요해!.ipynb)

path = r' 여러분이 내려 받은 KB 엑셀파일의 디렉터리를 넣으세요 \ KB엑셀 파일명.xls'
wb = xw.Book(path)                
sheet = wb.sheets['PIR(월별)']   
row_num = sheet.range('J2').end('down').row  
data_range = 'B2:N' + str(row_num)
pir_rawdata = sheet[data_range].options(pd.DataFrame, index=False, header=True).value 

In [ ]:
# [예제 2.34] 지역을 구분하는 상위 컬럼 만들기 

big_col = list(pir_rawdata.columns)
big_col[0] = 'index1'
big_col[1] = 'index2'
big_col[2] = 'index3'

for num, col in enumerate(big_col):
    if col == None:
        big_col[num] = big_col[num - 1]
    else:
        pass

In [ ]:
# [예제 2.35] 소득 분위를 나타내는 하위 컬럼 만들기 

small_col = list(pir_rawdata.loc[1])
small_col[0] = 'index1'
small_col[1] = 'index2'
small_col[2] = 'index3'

In [ ]:
# [예제 2.36] 이중 컬럼 설정 

pir_rawdata.columns = [big_col, small_col]
pir_rawdata.drop([0,1], inplace=True)

In [ ]:
# [예제 2.37] 상위 인덱스 설정

big_index = list(pir_rawdata['index1']['index1'])
for num, index in enumerate(big_index):
    if index is not None:
        if type(index) == str:
            big_index[num] = '20' + index.split(".")[0][1:] + '.' + index.split(".")[1][:2]
        else:
            big_index[num] = big_index[num - 1].split(".")[0] + "." +  str(int(index))
    else:
        big_index[num] = big_index[num - 1]

In [ ]:
# [예제 2.38] 하위 인덱스 및 이중 인덱스 설정  

small_index = list(pir_rawdata['index3']['index3'])
pir_rawdata.index = [pd.to_datetime(big_index), small_index]

del pir_rawdata['index1']
del pir_rawdata['index2']
del pir_rawdata['index3']

pir_rawdata.index.names = ['날짜', '평균주택가격']

In [ ]:
pir_rawdata.xs('3분위', level='평균주택가격')

In [ ]:
# [예제 2.39] 서울 각 소득 분위의 3분위 주택가격 시계열 subplot으로 나타내기  

gagus = ['1분위', '2분위', '3분위', '4분위', '5분위']
location = '서울 Seoul'
num_row = int((len(gagus)-1)/2)+1

plt.figure(figsize=(10, num_row*5))
for i, gagu in enumerate(gagus):
    plt.subplot(num_row, 2, i+1)
    plt.title(gagu + " 가구의 중간값(3분위) 주택가격 PIR")
    plt.plot(pir_rawdata.xs('3분위', level='평균주택가격')[location][gagu])
    
plt.show()

In [ ]:
pir_rawdata

In [ ]:
# [예제 2.40] PIR 그래프에 평균값 추가하기 

gagus = ['1분위', '2분위', '3분위', '4분위', '5분위']
location = '서울 Seoul'
house_price_level = '3분위'
num_row = int((len(gagus)-1)/2)+1

plt.figure(figsize=(10, num_row*5))
for i, gagu in enumerate(gagus):
    plt.subplot(num_row, 2, i+1)
    plt.title(gagu + " 가구의 중간값(" + house_price_level + ") 주택가격 PIR")
    plt.plot(pir_rawdata.xs(house_price_level, level='평균주택가격')[location][gagu])
    indx = pir_rawdata.xs(house_price_level, level='평균주택가격')[location][gagu].index
    long_mean = pir_rawdata.xs(house_price_level, level='평균주택가격')[location][gagu].mean()
    plt.plot(indx, [long_mean for a in range(len(pir_rawdata.xs(house_price_level, level='평균주택가격')[location][gagu]))])
    
plt.show()